In [1]:
import torch

In [20]:
#tensor of 3 dimensions.
#0th dim: 2 elements (z-dim)
#1st dim: 4 elements (y-dim)
#2nd dim: 5 elements (x-dim, inner most dimension)

#
torch.randn(2,3,4,5)

tensor([[[[ 2.6389e-01,  1.1832e+00,  5.9789e-01, -4.7175e-01, -1.5782e+00],
          [ 9.7796e-01,  6.8290e-01,  1.1438e+00, -3.4283e-01,  9.2105e-01],
          [ 1.6242e+00, -5.5182e-01, -1.4361e+00,  8.6247e-01, -7.0251e-01],
          [-1.3000e+00,  6.5496e-01, -4.0667e-01,  5.1401e-01, -6.0364e-01]],

         [[ 8.9855e-01,  6.4206e-01,  1.2591e+00,  1.3681e+00,  2.2861e+00],
          [ 6.6919e-01, -1.3948e+00,  1.6706e-01,  1.7152e+00, -8.4802e-01],
          [ 7.3046e-01, -1.8687e+00, -7.7028e-02, -9.5230e-01, -3.7267e-01],
          [ 1.7549e+00, -7.6685e-01,  1.3811e+00,  1.7188e+00,  1.4548e+00]],

         [[ 9.2964e-02, -3.3725e-01, -1.0614e+00,  3.7704e-01, -4.3399e-01],
          [ 1.1145e+00, -2.2078e-01, -1.3296e+00,  1.4322e-01,  7.6692e-01],
          [ 1.2338e+00, -8.4860e-01,  4.7482e-01, -9.5463e-01,  5.1414e-01],
          [-6.9946e-01,  3.8275e-01, -4.8985e-01, -9.6242e-01,  2.3880e+00]]],


        [[[-1.1462e+00, -9.1014e-01,  2.0079e-01,  1.0401e-01,  8.10